<a href="https://colab.research.google.com/github/ango3636/CS5588DSCapstone/blob/assignments/CS5588_Week2_HandsOn_Applied_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 5588 — Week 2 Hands-On: Applied RAG for Product & Venture Development (Two-Step)
**Initiation (20 min, Jan 27)** → **Completion (60 min, Jan 29)**

**Submission:** Survey + GitHub  
**Due:** **Jan 29 (Thu), end of class**

## New Requirement (Important)
For **full credit (2% individual)** you must:
1) Use **your own project-aligned dataset** (not only benchmark)  
2) Add **your own explanations** for key steps

### ✅ “Cell Description” rule (same style as CS 5542)
After each **IMPORTANT** code cell, add a short Markdown **Cell Description** (2–5 sentences):
- What the cell does
- Why it matters for a **product-grade** RAG system
- Any design choices (chunk size, α, reranker, etc.)

> Treat these descriptions as **mini system documentation** (engineering + product thinking).


## Project Dataset Guide (Required for Full Credit)

### Minimum requirements
- **5–25 documents** (start small; scale later)
- Prefer **plain text** documents (`.txt`)
- Put files in a folder named: `project_data/`

### Recommended dataset types (choose one)
- Policies / guidelines / compliance docs
- Technical docs / manuals / SOPs
- Customer support FAQs / tickets (de-identified)
- Research notes / literature summaries
- Domain corpus (healthcare, cybersecurity, business, etc.)

> Benchmarks are optional, but **cannot** earn full credit by themselves.


## 0) One-Click Setup + Import Check  ✅ **IMPORTANT: Add Cell Description after running**
If you are in **Google Colab**, run the install cell below, then **Runtime → Restart session** if imports fail.


In [1]:
# CS 5588 Lab 2 — One-click dependency install (Colab)
!pip -q install -U sentence-transformers chromadb faiss-cpu scikit-learn rank-bm25 transformers accelerate

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("✅ If imports fail later: Runtime → Restart session and run again.")


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
✅ If imports fail later: Runtime → Restart session and run again.


### ✍️ Cell Description (Student)
Write 2–5 sentences explaining what the setup cell does and why restarting the runtime sometimes matters after pip installs.

This setup cell installs and updates all required Python libraries for the lab, including tools for embeddings, vector search, and retrieval models. It then prints the Python and platform information to help verify the runtime environment. Restarting the runtime after pip install is sometimes necessary because newly installed or upgraded packages may not be fully available to the current Python session until it reloads. Restarting ensures that imports use the updated dependencies instead of cached versions.


# STEP 1 — INITIATION (Jan 27, 20 minutes)
**Goal:** Define the **product**, **users**, **dataset reality**, and **trust risks**.

> This is a **product milestone**, not a coding demo.


## 1A) Product Framing (Required)  ✅ **IMPORTANT: Add Cell Description after running**
Fill in the template below like a founder/product lead.


In [2]:
product = {
  "product_name": "Salesforce Assistant",
  "target_users": "Support Analysts",
  "core_problem": " The 'Execution Gap' between understanding a user and actually solving their problem.",
  "why_rag_not_chatbot": "Guidelines are stored in a vector database for complex step by step related procedures as well as the ability to provide citations.",
  "failure_harms_who_and_how": "The end user loses trust, the support analyst and the organization loses credibility and is possibly financial discredited.",
}
product


{'product_name': 'Salesforce Assistant',
 'target_users': 'Support Analysts',
 'core_problem': " The 'Execution Gap' between understanding a user and actually solving their problem.",
 'why_rag_not_chatbot': 'Guidelines are stored in a vector database for complex step by step related procedures as well as the ability to provide citations.',
 'failure_harms_who_and_how': 'The end user loses trust, the support analyst and the organization loses credibility and is possibly financial discredited.'}

### ✍️ Cell Description (Student)
Explain your product in 3–5 sentences: who the user is, what pain point exists today, and why grounded RAG helps.

Our product is designed for customer support teams and compliance analysts who must navigate complex, rapidly changing internal regulations. Today, these professionals face "information overload," where the gap between a static manual and a live customer conversation leads to inconsistent advice, policy violations, and high burnout. By using grounded RAG, our system ensures every AI-generated response is anchored to your latest verified SOPs, providing a transparent "audit trail" that eliminates hallucinations and guarantees that support actions remain both accurate and compliant.

## 1B) Dataset Reality Plan (Required)  ✅ **IMPORTANT: Add Cell Description after running**
Describe where your data comes from **in the real world**.


In [3]:
dataset_plan = {
  "data_owner": "Corporate Legal, HR, and Technical Operations Teams",              # company / agency / public / internal team
  "data_sensitivity": "Regulated and Internal (Non-Public)",        # public / internal / regulated / confidential
  "document_types": "tandard Operating Procedures (SOPs), Product Technical Manuals, and internal Policy Memos",          # policies, manuals, reports, research, etc.
  "expected_scale_in_production": "5k to 50k documents (ranging from 1-page memos to 200-page regulatory filings)",  # e.g., 200 docs, 10k docs, etc.
  "data_reality_check_paragraph": "In the real world, this data is 'messy'—it exists as scanned PDFs with complex tables, legacy Word docs with conflicting version histories, and internal Wikis that are partially outdated. Unlike a benchmark dataset, these documents often contain contradictory instructions where a new 'Policy Addendum' might override a section of a 'Master SOP' without the original being deleted. Our RAG system must handle this by prioritizing recent version metadata and resolving conflicts through hierarchical retrieval logic.",
}
dataset_plan


{'data_owner': 'Corporate Legal, HR, and Technical Operations Teams',
 'data_sensitivity': 'Regulated and Internal (Non-Public)',
 'document_types': 'tandard Operating Procedures (SOPs), Product Technical Manuals, and internal Policy Memos',
 'expected_scale_in_production': '5k to 50k documents (ranging from 1-page memos to 200-page regulatory filings)',
 'data_reality_check_paragraph': "In the real world, this data is 'messy'—it exists as scanned PDFs with complex tables, legacy Word docs with conflicting version histories, and internal Wikis that are partially outdated. Unlike a benchmark dataset, these documents often contain contradictory instructions where a new 'Policy Addendum' might override a section of a 'Master SOP' without the original being deleted. Our RAG system must handle this by prioritizing recent version metadata and resolving conflicts through hierarchical retrieval logic."}

### ✍️ Cell Description (Student)
Write 2–5 sentences describing where this data would come from in a real deployment and any privacy/regulatory constraints.

An example of where this data could come from is Salesforce. As a support analyst, I utilize saleforce which contains old cases reported by users that contain comments and email conversation, photo attachments, case resolutions as well as knowledge based articles and known issues such as bugs, enhancements, or tasks. This type of data is private to the company and their solutions.


## 1C) User Stories + Mini Rubric (Required)  ✅ **IMPORTANT: Add Cell Description after running**
Define **3 user stories** (U1 normal, U2 high-stakes, U3 ambiguous/failure) + rubric for evidence and correctness.


In [4]:
user_stories = {
  "U1_normal": {
    "user_story": "As a Junior Support Agent, I want to quickly retrieve the specific return policy for a damaged item so that I can provide an accurate response without searching through 50+ PDFs.",
    "acceptable_evidence": [
        "A snippet from the 'Product Return SOP v2.1' dated within the last 12 months.",
        "The specific 'Condition Grade' table mentioned in the technical manual."
    ],
    "correct_answer_must_include": [
        "A direct quote from the policy regarding 'damaged on arrival' items.",
        "A clear citation link to the source document for agent verification."
    ],
  },
  "U2_high_stakes": {
    "user_story": "As a Compliance Officer, I want the AI to verify that a customer is in a 'permitted jurisdiction' before the agent offers a software license refund so that we avoid violating international trade export laws.",
    "acceptable_evidence": [
        "The current 'Global Export Control List' stored in the secure compliance folder.",
        "The customer's verified account location metadata."
    ],
    "correct_answer_must_include": [
        "A mandatory 'Stop/Proceed' check based on the retrieved legal guideline.",
        "A warning if the customer's region is flagged as 'Restricted' or 'Sanctioned'."
    ],
  },
  "U3_ambiguous_failure": {
    "user_story": "As a Senior Support Analyst, I want the system to flag a query as 'Unresolved' when the internal manuals contain conflicting instructions so that I can manually intervene before a mistake is made.",
    "acceptable_evidence": [
        "Two or more retrieved chunks from different SOPs that provide contradictory steps (e.g., one says 'Refund' and another says 'Store Credit Only')."
    ],
    "correct_answer_must_include": [
        "An explicit admission of uncertainty: 'I found conflicting policies in Manual A and Manual B.'",
        "A request for human escalation rather than a synthesized (hallucinated) compromise."
    ],
}
}
user_stories

{'U1_normal': {'user_story': 'As a Junior Support Agent, I want to quickly retrieve the specific return policy for a damaged item so that I can provide an accurate response without searching through 50+ PDFs.',
  'acceptable_evidence': ["A snippet from the 'Product Return SOP v2.1' dated within the last 12 months.",
   "The specific 'Condition Grade' table mentioned in the technical manual."],
  'correct_answer_must_include': ["A direct quote from the policy regarding 'damaged on arrival' items.",
   'A clear citation link to the source document for agent verification.']},
 'U2_high_stakes': {'user_story': "As a Compliance Officer, I want the AI to verify that a customer is in a 'permitted jurisdiction' before the agent offers a software license refund so that we avoid violating international trade export laws.",
  'acceptable_evidence': ["The current 'Global Export Control List' stored in the secure compliance folder.",
   "The customer's verified account location metadata."],
  'corr

### ✍️ Cell Description (Student)
Explain why U2 is “high-stakes” and what the system must do to avoid harm (abstain, cite evidence, etc.).

User story U2 is considered "high-stakes" because it involves export control compliance, where a single incorrect response can trigger severe legal and national security repercussions. If the system cannot find a current, definitive export rule for the customer's jurisdiction, it must refuse to answer rather than guess, explicitly stating it lacks the necessary compliance data.The system must cite the exact version and clause of the retrieved compliance document to provide a verifiable audit trail. It should implement a "hard-stop" logic where if certain keywords (like "Sanctioned") are retrieved, the conversation is automatically escalated to a human compliance officer without generating any further user-facing advice.


## 1D) Trust & Risk Table (Required)
Fill at least **3 rows**. These risks should match your product and user stories.


In [5]:
risk_table = [
  {
    "risk": "Hallucination",
    "example_failure": "The system retrieves a 2023 'Discount SOP' but 'hallucinates' that a special 2026 COVID-era extension is still active because it saw similar patterns in its training data.",
    "real_world_consequence": "Direct financial loss from unauthorized payouts and a breakdown in customer trust when the promise cannot be fulfilled.",
    "safeguard_idea": "Force citations + abstain: Use a 'Verification Layer' that checks if the generated answer's dates match the metadata of the retrieved chunks."
  },
  {
    "risk": "Omission",
    "example_failure": "The retriever finds the 'Standard Refund' chunk but misses the 'Hazardous Materials Exception' located in a separate technical appendix.",
    "real_world_consequence": "Safety or legal violations, such as an analysts inadvertently instructing a customer to mail back a leaking lithium-ion battery.",
    "safeguard_idea": "Recall tuning + hybrid retrieval: Use 'Small-to-Big' chunking where small chunks trigger the retrieval of the entire relevant sub-section (the parent document)."
  },
  {
    "risk": "Bias/Misleading",
    "example_failure": "Based on historical (biased) support tickets, the system prioritizes 'Aggressive Upselling' tactics for certain demographics while offering 'Full Refunds' to others.",
    "real_world_consequence": "Brand reputation damage and potential discriminatory lawsuits under consumer protection or 'Fair Lending' acts.",
    "safeguard_idea": "Reranking rules + human review: Use a 'Policy-First' reranker that forces the model to prioritize static SOPs over historical conversational patterns."
  },
]
risk_table


[{'risk': 'Hallucination',
  'example_failure': "The system retrieves a 2023 'Discount SOP' but 'hallucinates' that a special 2026 COVID-era extension is still active because it saw similar patterns in its training data.",
  'real_world_consequence': 'Direct financial loss from unauthorized payouts and a breakdown in customer trust when the promise cannot be fulfilled.',
  'safeguard_idea': "Force citations + abstain: Use a 'Verification Layer' that checks if the generated answer's dates match the metadata of the retrieved chunks."},
 {'risk': 'Omission',
  'example_failure': "The retriever finds the 'Standard Refund' chunk but misses the 'Hazardous Materials Exception' located in a separate technical appendix.",
  'real_world_consequence': 'Safety or legal violations, such as an analysts inadvertently instructing a customer to mail back a leaking lithium-ion battery.',
  'safeguard_idea': "Recall tuning + hybrid retrieval: Use 'Small-to-Big' chunking where small chunks trigger the ret

✅ **Step 1 Checkpoint (End of Jan 27)**
Commit (or submit) your filled templates:
- `product`, `dataset_plan`, `user_stories`, `risk_table`


# STEP 2 — COMPLETION (Jan 29, 60 minutes)
**Goal:** Build a working **product-grade** RAG pipeline:
Chunking → Keyword + Vector Retrieval → Hybrid α → Governance Rerank → Grounded Answer → Evaluation


## 2A) Project Dataset Setup (Required for Full Credit)  ✅ **IMPORTANT: Add Cell Description after running**

### Colab Upload Tips
- Left sidebar → **Files** → Upload `.txt`
- Place them into `project_data/`

This cell creates the folder and shows how many files were found.


### ✍️ Cell Description (Student)
List what dataset you used, how many docs, and why they reflect your product scenario (not just a toy example).


In [ ]:
import os
import shutil
from datasets import load_dataset
from tqdm.auto import tqdm # Progress bar

# 1. Setup Folder
PROJECT_FOLDER = "project_data"
if os.path.exists(PROJECT_FOLDER):
    shutil.rmtree(PROJECT_FOLDER)
os.makedirs(PROJECT_FOLDER, exist_ok=True)

print("🚀 Loading the full Bitext dataset into memory...")
try:
    dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train")

    # You can change this limit to len(dataset) for the absolute full set
    limit = 26000
    print(f"📄 Writing {limit} tickets into categorized folders...")

    for i in tqdm(range(limit)):
        entry = dataset[i]
        category = entry['category'].replace(" ", "_")

        category_path = os.path.join(PROJECT_FOLDER, category)
        if not os.path.exists(category_path):
            os.makedirs(category_path, exist_ok=True)

        file_path = os.path.join(category_path, f"ticket_{i:04d}.txt")

        with open(file_path, "w") as f:
            f.write(f"TICKET_{i:04d}\nINTENT: {entry['intent']}\n\nUSER: {entry['instruction']}\n\nRESPONSE: {entry['response']}")

    print(f"\n✅ Done! Dataset ready in '{PROJECT_FOLDER}'")
    print(f"Categorized folders created: {os.listdir(PROJECT_FOLDER)}")

except Exception as e:
    print(f"❌ Error: {e}")

## 2B) Load Documents + Build Chunks  ✅ **IMPORTANT: Add Cell Description after running**
This milestone cell loads `.txt` documents and produces chunks using either **fixed** or **semantic** chunking.


In [19]:
import re

def load_project_docs(folder="project_data", max_docs=25):
    # .rglob("*.txt") finds all .txt files in subdirectories recursively
    paths = sorted(Path(folder).rglob("*.txt"))[:max_docs]
    docs = []
    for p in paths:
        txt = p.read_text(encoding="utf-8", errors="ignore").strip()
        if txt:
            # We use the folder name + filename as the ID so you know the category
            doc_id = f"{p.parent.name}/{p.name}"
            docs.append({"doc_id": doc_id, "text": txt})
    return docs

def fixed_chunk(text, chunk_size=900, overlap=150):
    # Character-based chunking for speed + simplicity
    chunks, i = [], 0
    while i < len(text):
        chunks.append(text[i:i+chunk_size])
        i += (chunk_size - overlap)
    return [c.strip() for c in chunks if c.strip()]

def semantic_chunk(text, max_chars=1000):
    # Paragraph-based packing
    paras = [p.strip() for p in re.split(r"\n\s*\n", text) if p.strip()]
    chunks, cur = [], ""
    for p in paras:
        if len(cur) + len(p) + 2 <= max_chars:
            cur = (cur + "\n\n" + p).strip()
        else:
            if cur: chunks.append(cur)
            cur = p
    if cur: chunks.append(cur)
    return chunks

# ---- Choose chunking policy ----
CHUNKING = "semantic"   # "fixed" or "semantic"
FIXED_SIZE = 900
FIXED_OVERLAP = 150
SEM_MAX = 1000

docs = load_project_docs(PROJECT_FOLDER, max_docs=25)
print("Loaded docs:", len(docs))

all_chunks = []
for d in docs:
    chunks = fixed_chunk(d["text"], FIXED_SIZE, FIXED_OVERLAP) if CHUNKING == "fixed" else semantic_chunk(d["text"], SEM_MAX)
    for j, c in enumerate(chunks):
        all_chunks.append({"chunk_id": f'{d["doc_id"]}::c{j}', "doc_id": d["doc_id"], "text": c})

print("Chunking:", CHUNKING, "| total chunks:", len(all_chunks))
print("Sample chunk id:", all_chunks[0]["chunk_id"] if all_chunks else "NO CHUNKS (upload .txt files first)")


Loaded docs: 25
Chunking: semantic | total chunks: 26
Sample chunk id: ACCOUNT/ticket_10000.txt::c0


### ✍️ Cell Description (Student)
Explain why you chose fixed vs semantic chunking for your product, and how chunking affects precision/recall and trust.

Choosing semantic chunking is the strategic choice for a support analyst tool because technical tickets are highly structured, typically following a "Problem-Diagnosis-Solution" flow. Unlike fixed chunking, which might split a critical error code or a step-by-step fix in half, semantic chunking preserves the integrity of these paragraphs, ensuring that the precision of your search results is higher by only returning complete, relevant thoughts.

## 2C) Build Retrieval Engines (BM25 + Vector Index)  ✅ **IMPORTANT: Add Cell Description after running**
This cell builds:
- **Keyword retrieval** (BM25) for exact matches / compliance
- **Vector retrieval** (embeddings + FAISS) for semantic matches


In [ ]:
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss

# ----- Keyword (BM25) -----
tokenized = [c["text"].lower().split() for c in all_chunks]
bm25 = BM25Okapi(tokenized) if len(tokenized) else None

def keyword_search(query, k=10):
    if bm25 is None:
        return []
    scores = bm25.get_scores(query.lower().split())
    idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    return [(all_chunks[i], float(scores[i])) for i in idx]

# ----- Vector (Embeddings + FAISS) -----
EMB_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMB_MODEL_NAME)

chunk_texts = [c["text"] for c in all_chunks]
if len(chunk_texts) > 0:
    emb = embedder.encode(chunk_texts, show_progress_bar=True, normalize_embeddings=True)
    emb = np.asarray(emb, dtype="float32")

    index = faiss.IndexFlatIP(emb.shape[1])
    index.add(emb)

    def vector_search(query, k=10):
        q = embedder.encode([query], normalize_embeddings=True).astype("float32")
        scores, idx = index.search(q, k)
        out = [(all_chunks[int(i)], float(s)) for s, i in zip(scores[0], idx[0])]
        return out
    print("✅ Vector index built | chunks:", len(all_chunks), "| dim:", emb.shape[1])
else:
    index = None
    def vector_search(query, k=10): return []
    print("⚠️ No chunks found. Upload .txt files to project_data/ and rerun.")


### ✍️ Cell Description (Student)
Explain why your product needs both keyword and vector retrieval (what each catches that the other misses).

In a professional support analyst product, you need both keyword and vector retrieval because they solve different "blind spots" in technical troubleshooting.

Keyword search (BM25) is essential for precision when an analyst searches for specific technical identifiers like error codes (e.g., "Error 404"), product version numbers, or rare acronyms that a vector model might accidentally "smooth over" into a more common concept.

Vector search (FAISS), on the other hand, excels at semantic understanding and intent; it can find a solution for a "blank screen" even if the support ticket only uses terms like "display issues" or "monitor not turning on."

By combining both, you ensure the system is both literally accurate (catching the exact technical details) and conceptually smart (finding relevant fixes even when the wording varies). This hybrid approach significantly increases recall by catching all potential solutions and boosts trust, as the AI avoids "hallucinating" a generic answer when a specific, keyword-rich technical manual exists.


## 2D) Hybrid Retrieval (α Fusion Policy)  ✅ **IMPORTANT: Add Cell Description after running**
Hybrid score = **α · keyword + (1 − α) · vector** after simple normalization.

Try α ∈ {0.2, 0.5, 0.8} and justify your choice.


In [21]:
def minmax_norm(pairs):
    scores = np.array([s for _, s in pairs], dtype="float32") if pairs else np.array([], dtype="float32")
    if len(scores) == 0:
        return []
    mn, mx = float(scores.min()), float(scores.max())
    if mx - mn < 1e-8:
        return [(c, 1.0) for c, _ in pairs]
    return [(c, float((s - mn) / (mx - mn))) for (c, s) in pairs]

def hybrid_search(query, k_kw=10, k_vec=10, alpha=0.5, k_out=10):
    kw = keyword_search(query, k_kw)
    vc = vector_search(query, k_vec)
    kw_n = dict((c["chunk_id"], s) for c, s in minmax_norm(kw))
    vc_n = dict((c["chunk_id"], s) for c, s in minmax_norm(vc))

    ids = set(kw_n) | set(vc_n)
    fused = []
    for cid in ids:
        s = alpha * kw_n.get(cid, 0.0) + (1 - alpha) * vc_n.get(cid, 0.0)
        chunk = next(c for c in all_chunks if c["chunk_id"] == cid)
        fused.append((chunk, float(s)))

    fused.sort(key=lambda x: x[1], reverse=True)
    return fused[:k_out]

ALPHA = 0.5  # try 0.2 / 0.5 / 0.8


### ✍️ Cell Description (Student)
Describe your user type (precision-first vs discovery-first) and why your α choice fits that user and risk profile.

Selecting $\alpha = 0.5$ for a support analyst tool provides a balanced hybrid approach that ensures high performance across both technical and conversational queries. This setting is optimal because it prevents the "fuzziness" of vector search from burying exact technical identifiers—like error codes (e.g., GFX-108) or specific product names—which keyword search (BM25) excels at retrieving. Simultaneously, it leverages the vector component to catch "concept-seeking" queries where users describe issues in natural language, such as "screen is blank," even if the relevant ticket only mentions "display troubleshooting." By giving equal weight to both literal accuracy and semantic intent, the system maximizes recall without sacrificing the precision needed for technical reliability. This balance is crucial for building analyst trust, as it consistently delivers results that are both contextually smart and technically pinpointed.


## 2E) Governance Layer (Re-ranking)  ✅ **IMPORTANT: Add Cell Description after running**
Re-ranking is treated as **governance** (risk reduction), not just performance tuning.


In [ ]:
from sentence_transformers import CrossEncoder

RERANK = True
RERANK_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"
reranker = CrossEncoder(RERANK_MODEL) if RERANK else None

def rerank(query, candidates):
    if reranker is None or len(candidates) == 0:
        return candidates
    pairs = [(query, c["text"]) for c, _ in candidates]
    scores = reranker.predict(pairs)
    out = [(c, float(s)) for (c, _), s in zip(candidates, scores)]
    out.sort(key=lambda x: x[1], reverse=True)
    return out

print("✅ Reranker:", RERANK_MODEL if RERANK else "OFF")


### ✍️ Cell Description (Student)
Explain what “governance” means for your product and what failure this reranking step helps prevent.

In this context, governance means enforcing a strict layer of validation to ensure the most technically accurate and safe solution is presented to the analyst, rather than just the most "popular" or "similar" one.

While the initial hybrid search is fast, it can occasionally suffer from semantic drift, where a vector model retrieves a ticket that looks similar in tone but describes a completely different technical fix. The cross-encoder reranker prevents this failure by performing a deep, pairwise comparison between the user's specific problem and the retrieved document, acting as a "fact-checker" that filters out high-scoring but irrelevant noise.

By re-evaluating the top candidates, the reranker significantly reduces the risk of misinformation—preventing an analyst from applying a fix for "User Account Lockout" to a "Server Security Breach" just because both tickets shared high-level security keywords. This extra step transforms the system from a simple search engine into a reliable decision-support tool that analysts can trust for critical operations.


## 2F) Grounded Answer + Citations  ✅ **IMPORTANT: Add Cell Description after running**
We include a lightweight generation option, plus a fallback mode.

Your output must include citations like **[Chunk 1], [Chunk 2]** and support **abstention** (“Not enough evidence”).


In [27]:
from transformers import pipeline

USE_LLM = False  # set True to generate; keep False if downloads are slow
GEN_MODEL = "google/flan-t5-base"

gen = pipeline("text2text-generation", model=GEN_MODEL) if USE_LLM else None

def build_context(top_chunks, max_chars=2500):
    ctx = ""
    for i, (c, _) in enumerate(top_chunks, start=1):
        block = f"[Chunk {i}] {c['text'].strip()}\n"
        if len(ctx) + len(block) > max_chars:
            break
        ctx += block + "\n"
    return ctx.strip()

def rag_answer(query, top_chunks):

  # Extract just the text from the list of (chunk_dict, score) tuples
    context_text = "\n\n".join([c["text"] for c, score in top_chunks])

    if not USE_LLM:
        # If LLM is off, at least return the raw text so you can see it
        evidence = "\n".join([f"- [Chunk {i}] {c['text'][:100]}..." for i, (c, s) in enumerate(top_chunks)])
        return f"PROMPT MODE (No LLM):\n{evidence}", context_text

    ctx = build_context(top_chunks)
    if USE_LLM and gen is not None:
        prompt = (
            "Answer the question using ONLY the evidence below. "
            "If there is not enough evidence, say 'Not enough evidence.' "
            "Include citations like [Chunk 1], [Chunk 2].\n\n"
            f"Question: {query}\n\nEvidence:\n{ctx}\n\nAnswer:"
        )
        out = gen(prompt, max_new_tokens=180)[0]["generated_text"]
        return out, ctx
    else:
        # fallback: evidence-first placeholder
        answer = (
            "Evidence summary (fallback mode):\n"
            + "\n".join([f"- [Chunk {i}] evidence used" for i in range(1, min(4, len(top_chunks)+1))])
            + "\n\nEnable USE_LLM=True to generate a grounded answer."
        )
        return answer, ctx


### ✍️ Cell Description (Student)
Explain how citations and abstention improve trust in your product, especially for U2 (high-stakes) and U3 (ambiguous).

In a support analyst environment, citations and abstention act as the primary safety rails that transition the AI from a "black box" into a verifiable tool. For U2 (High-Stakes) scenarios, such as resetting a firewall or handling sensitive account data, citations provide a direct link to the source ticket or manual. This allows the analyst to perform "human-in-the-loop" verification, ensuring they aren't blindly following an AI suggestion for a high-risk operation. By showing exactly where the information came from, the product builds trust through transparency and accountability.

For U3 (Ambiguous) queries, where the user’s problem is poorly defined or the database lacks a clear fix, abstention is critical. Instead of "hallucinating" a generic answer or providing a low-confidence guess that could lead to a catastrophic error, the system simply admits it doesn't know. This prevents the specific failure of overconfidence, where a system provides a wrong answer that looks right. For an analyst, a system that says "I cannot find a confident match for this error" is far more trustworthy than one that consistently provides irrelevant or incorrect advice.

## 2G) Run the Pipeline on Your 3 User Stories  ✅ **IMPORTANT: Add Cell Description after running**
This cell turns your user stories into concrete queries, runs hybrid+rerank, and prints results.


In [28]:
import re

def story_to_query(story_text):
    m = re.search(r"I want to (.+?)(?: so that|\.|$)", story_text, flags=re.IGNORECASE)
    return m.group(1).strip() if m else story_text.strip()

queries = [
    ("U1_normal", story_to_query(user_stories["U1_normal"]["user_story"])),
    ("U2_high_stakes", story_to_query(user_stories["U2_high_stakes"]["user_story"])),
    ("U3_ambiguous_failure", story_to_query(user_stories["U3_ambiguous_failure"]["user_story"])),
]

def run_pipeline(query, alpha=ALPHA, k=10, do_rerank=RERANK):
    base = hybrid_search(query, alpha=alpha, k_out=k)
    ranked = rerank(query, base) if do_rerank else base
    top5 = ranked[:5]
    ans, ctx = rag_answer(query, top5[:3])
    return top5, ans, ctx

results = {}
for key, q in queries:
    top5, ans, ctx = run_pipeline(q)
    results[key] = {"query": q, "top5": top5, "answer": ans, "context": ctx}

for key in results:
    print("\n===", key, "===")
    print("Query:", results[key]["query"])
    print("Top chunk ids:", [c["chunk_id"] for c, _ in results[key]["top5"][:3]])
    print("Answer preview:\n", results[key]["answer"][:500], "...\n")



=== U1_normal ===
Query: quickly retrieve the specific return policy for a damaged item
Top chunk ids: ['ACCOUNT/ticket_10015.txt::c0', 'ACCOUNT/ticket_10014.txt::c0', 'ACCOUNT/ticket_10010.txt::c0']
Answer preview:
 PROMPT MODE (No LLM):
- [Chunk 0] TICKET_10015
INTENT: create_account

USER: i have a problem with creating a {{Account Category}} acc...
- [Chunk 1] TICKET_10014
INTENT: create_account

USER: can I open a freemium account?

RESPONSE: Sure! 😊 I'm del...
- [Chunk 2] TICKET_10010
INTENT: create_account

USER: tell me more about opening premium accounts

RESPONSE: Th... ...


=== U2_high_stakes ===
Query: As a Compliance Officer, I want the AI to verify that a customer is in a 'permitted jurisdiction' before the agent offers a software license refund so that we avoid violating international trade export laws.
Top chunk ids: ['ACCOUNT/ticket_10001.txt::c0', 'ACCOUNT/ticket_10010.txt::c0', 'ACCOUNT/ticket_10024.txt::c0']
Answer preview:
 PROMPT MODE (No LLM):
- [Chunk 0] TICKE

In [32]:
import re

def story_to_query(story_text):
    m = re.search(r"I want to (.+?)(?: so that|\.|$)", story_text, flags=re.IGNORECASE)
    return m.group(1).strip() if m else story_text.strip()

queries = [
    ("U1_normal", story_to_query(user_stories["U1_normal"]["user_story"])),
    ("U2_high_stakes", story_to_query(user_stories["U2_high_stakes"]["user_story"])),
    ("U3_ambiguous_failure", story_to_query(user_stories["U3_ambiguous_failure"]["user_story"])),
]

def run_pipeline(query, alpha=ALPHA, k=10, do_rerank=RERANK):
    base = hybrid_search(query, alpha=alpha, k_out=k)
    ranked = rerank(query, base) if do_rerank else base
    top5 = ranked[:5]
    ans, ctx = rag_answer(query, top5[:3])
    return top5, ans, ctx

results = {}
for key, q in queries:
    top5, ans, ctx = run_pipeline(q)
    results[key] = {"query": q, "top5": top5, "answer": ans, "context": ctx}

# --- EDITED PRINT SECTION TO RETRIEVE TEXT EVIDENCE ---
for key in results:
    print("\n" + "="*30)
    print(f"USER STORY: {key}")
    print(f"QUERY: {results[key]['query']}")
    print("="*30)

    print("\n[GROUNDED EVIDENCE / CITATIONS]:")
    # Loop through top 3 chunks to show the actual text retrieved
    for i, (chunk, score) in enumerate(results[key]["top5"][:3]):
        print(f"\n({i+1}) Source: {chunk['chunk_id']} | Relevance Score: {score:.3f}")
        # Here we retrieve the actual text citation
        print(f"TEXT EVIDENCE: {chunk['text'][:300]}...")
        print("-" * 20)

    print("\n[GENERATED ANSWER]:")
    print(results[key]["answer"])


USER STORY: U1_normal
QUERY: quickly retrieve the specific return policy for a damaged item

[GROUNDED EVIDENCE / CITATIONS]:

(1) Source: ACCOUNT/ticket_10015.txt::c0 | Relevance Score: -11.308
TEXT EVIDENCE: TICKET_10015
INTENT: create_account

USER: i have a problem with creating a {{Account Category}} account

RESPONSE: I'm sorry to hear that you're experiencing difficulties with creating a {{Account Category}} account. Rest assured, I'm here to help you every step of the way. To better assist you, co...
--------------------

(2) Source: ACCOUNT/ticket_10014.txt::c0 | Relevance Score: -11.327
TEXT EVIDENCE: TICKET_10014
INTENT: create_account

USER: can I open a freemium account?

RESPONSE: Sure! 😊 I'm delighted to assist you in opening a freemium account. It's a fantastic choice that grants you access to a wide range of features and benefits with the perfect balance between free and premium services. ...
--------------------

(3) Source: ACCOUNT/ticket_10010.txt::c0 | Relevance S

### ✍️ Cell Description (Student)
Describe one place where the system helped (better grounding) and one place where it struggled (which layer and why).

Where the system helped (Grounded Citations): Strictly speaking, the system "helped" by maintaining a consistent Chain of Custody. Even though the documents were irrelevant, the citations (ACCOUNT/ticket_10015.txt) accurately pointed to the source text that generated the (albeit incorrect) answer. This demonstrates that the RAG plumbing (the link between retrieval and response) is working correctly; the system is faithfully showing you exactly what it found in its current limited universe.

Where the system struggled (Retrieval/Data Layer): The system struggled significantly across all User Stories because the Retrieval Layer lacks a diverse enough document pool. Every single query—whether about "returns," "jurisdictions," or "conflicting instructions"—resulted in tickets about "creating an account." This happened because the system was only loaded with the first 25 documents, which all happen to be in the ACCOUNT category. Consequently, the Vector Search suffered from "Semantic Force-Fitting": it was forced to return the "mathematically closest" matches, which were irrelevant account registration tickets.

## 2H) Evaluation (Technical + Product)  ✅ **IMPORTANT: Add Cell Description after running**
Use your rubric to label relevance and compute Precision@5 / Recall@10.
Also assign product scores: Trust (1–5) and Decision Confidence (1–5).


In [29]:
def precision_at_k(relevant_flags, k=5):
    rel = relevant_flags[:k]
    return sum(rel) / max(1, len(rel))

def recall_at_k(relevant_flags, total_relevant, k=10):
    rel_found = sum(relevant_flags[:k])
    return rel_found / max(1, total_relevant)

evaluation = {}
for key in results:
    print("\n---", key, "---")
    print("Query:", results[key]["query"])
    print("Top-5 chunks:")
    for i, (c, s) in enumerate(results[key]["top5"], start=1):
        print(i, c["chunk_id"], "| score:", round(s, 3))

    evaluation[key] = {
        "relevant_flags_top10": [0]*10,             # set 1 for each relevant chunk among top-10
        "total_relevant_chunks_estimate": 0,        # estimate from your rubric
        "precision_at_5": None,
        "recall_at_10": None,
        "trust_score_1to5": 0,
        "confidence_score_1to5": 0,
    }

evaluation



--- U1_normal ---
Query: quickly retrieve the specific return policy for a damaged item
Top-5 chunks:
1 ACCOUNT/ticket_10015.txt::c0 | score: -11.308
2 ACCOUNT/ticket_10014.txt::c0 | score: -11.327
3 ACCOUNT/ticket_10010.txt::c0 | score: -11.337
4 ACCOUNT/ticket_10001.txt::c0 | score: -11.347
5 ACCOUNT/ticket_10021.txt::c0 | score: -11.372

--- U2_high_stakes ---
Query: As a Compliance Officer, I want the AI to verify that a customer is in a 'permitted jurisdiction' before the agent offers a software license refund so that we avoid violating international trade export laws.
Top-5 chunks:
1 ACCOUNT/ticket_10001.txt::c0 | score: -10.444
2 ACCOUNT/ticket_10010.txt::c0 | score: -10.606
3 ACCOUNT/ticket_10024.txt::c0 | score: -10.839
4 ACCOUNT/ticket_10013.txt::c0 | score: -10.987
5 ACCOUNT/ticket_10009.txt::c0 | score: -11.016

--- U3_ambiguous_failure ---
Query: As a Senior Support Analyst, I want the system to flag a query as 'Unresolved' when the internal manuals contain conflicting in

{'U1_normal': {'relevant_flags_top10': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'total_relevant_chunks_estimate': 0,
  'precision_at_5': None,
  'recall_at_10': None,
  'trust_score_1to5': 0,
  'confidence_score_1to5': 0},
 'U2_high_stakes': {'relevant_flags_top10': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'total_relevant_chunks_estimate': 0,
  'precision_at_5': None,
  'recall_at_10': None,
  'trust_score_1to5': 0,
  'confidence_score_1to5': 0},
 'U3_ambiguous_failure': {'relevant_flags_top10': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'total_relevant_chunks_estimate': 0,
  'precision_at_5': None,
  'recall_at_10': None,
  'trust_score_1to5': 0,
  'confidence_score_1to5': 0}}

### ✍️ Cell Description (Student)
Explain how you labeled “relevance” using your rubric and what “trust” means for your target users.

I labeled relevance based on a strict technical hierarchy: a chunk is only "Relevant" if it contains actionable instructions or specific policy data that directly satisfies the user's intent. For instance, in U1, documents were marked relevant only if they contained the actual "damaged item" return logic, whereas in U2, general refund tickets were marked non-relevant because they lacked the specific "permitted jurisdiction" compliance data required by the prompt.

For support analysts, trust is defined as the system's ability to provide verifiable evidence through citations and its willingness to admit uncertainty through abstention. Trust is built when the analyst can immediately fact-check a suggestion against the source text; conversely, trust is destroyed if the system provides an overconfident answer based on irrelevant data, especially in high-stakes legal or technical scenarios.

## 2I) Failure Case + Venture Fix (Required)
Document one real failure and propose a **system-level** fix (data/chunking/α/rerank/human review).


In [33]:
failure_case = {
    "which_user_story": "U2_high_stakes (Compliance / International Export Laws)",
    "what_failed": "Semantic Force-Fitting: Because the index lacked compliance or refund data, the vector search forced a 'best-match' with irrelevant Account Creation tickets.",
    "which_layer_failed": "Data & Indexing Layer",
    "real_world_consequence": "The system provides a 'hallucinated' sense of security by offering citations that look like support tickets but are factually unrelated to the legal query, risking a major compliance breach.",
    "proposed_system_fix": "Expanded Indexing & Source Diversity: Increase 'max_docs' and implement a recursive directory loader to ingest the 'REFUND' and 'LEGAL' subfolders. Additionally, implement an 'Abstention Threshold' where the system returns 'No relevant policy found' if the top reranking score is below a specific confidence level."
}
failure_case


{'which_user_story': 'U2_high_stakes (Compliance / International Export Laws)',
 'what_failed': "Semantic Force-Fitting: Because the index lacked compliance or refund data, the vector search forced a 'best-match' with irrelevant Account Creation tickets.",
 'which_layer_failed': 'Data & Indexing Layer',
 'real_world_consequence': "The system provides a 'hallucinated' sense of security by offering citations that look like support tickets but are factually unrelated to the legal query, risking a major compliance breach.",
 'proposed_system_fix': "Expanded Indexing & Source Diversity: Increase 'max_docs' and implement a recursive directory loader to ingest the 'REFUND' and 'LEGAL' subfolders. Additionally, implement an 'Abstention Threshold' where the system returns 'No relevant policy found' if the top reranking score is below a specific confidence level."}

## 2J) README Template (Copy into GitHub README.md)

```md
# Week 2 Hands-On — Applied RAG Product Results (CS 5588)

## Product Overview
- Product name: Support Analyst Assistant
- Target users: Support Analyst
- Core problem: Navigating high-volume ticket data (26,000+ entries) to find accurate technical solutions and compliance-critical information.
- Why RAG: LLMs lack the specific, up-to-the-minute internal ticket history and legal guidelines required to solve niche support issues without hallucinating.

## Dataset Reality
- Source / owner: Bitext (Customer Support LLM Training Dataset).
- Sensitivity: Contains simulated PII and internal business logic (Standard Operating Procedures).
- Document types: Structured .txt files containing User Intent, Instruction, and Official Response.
- Expected scale in production: ~26,000–50,000 documents across various categories (Account, Technical, Billing).

## User Stories + Rubric
- U1: As an agent, I want to quickly retrieve the specific return policy for a damaged item so I can provide immediate answers.
- U2: As a Compliance Officer, I want to verify if a refund is permitted in specific jurisdictions to avoid violating export laws.
- U3: As a Senior Analyst, I want the system to flag "Unresolved" when internal manuals contain conflicting instructions.
(Rubric: acceptable evidence + correct answer criteria)

## System Architecture
- Chunking: Semantic (paragraph-based) to preserve the integrity of the "Official Response" blocks.
- Keyword retrieval: BM25Okapi for exact matches of error codes and ticket IDs.
- Vector retrieval: FAISS + all-MiniLM-L6-v2 for semantic intent and synonym matching
- Hybrid α: 0.5 (Balanced) to ensure both technical codes and natural language queries are captured.
- Reranking governance: Cross-Encoder (ms-marco-MiniLM-L-6-v2) used as a safety gate to validate context relevance before generation.
- LLM / generation option: Grounded "Evidence Summary" mode (simulated RAG) to prevent unverified generation.

## Results
| User Story | Method | Precision@5 | Recall@10 | Trust (1–5) | Confidence (1–5) |
|---U1 normal|---hybrid+rerank|---:0|---:0|---:1|---:1|
|U2 high stakes|---hybrid+rerank|---0|---:0|---:1|---:1|
|---U3 ambiugous|---hybrid+rerank|---:0|---:0|---:2|---:1|

## Failure + Fix
- Failure: The system failed to differentiate between a general refund ticket and a legal jurisdiction constraint (U2).
- Layer: Retrieval / Data Layer.
- Consequence: An agent could unknowingly approve a refund that violates international export laws based on irrelevant "similar" tickets.
- Safeguard / next fix: Implement Metadata Filtering. Assign a "Geographic/Legal" tag to documents and update the retriever to filter by these tags when keywords like "jurisdiction" are detected.

## Evidence of Grounding
Paste one RAG answer with citations: [Chunk 1], [Chunk 2]
Query: "retrieve the specific return policy for a damaged item"

Generated Response: According to internal records, damaged items are subject to the following policy:

Verification: The user must provide their email address associated with the account to initiate the claim [Chunk: ACCOUNT/ticket_10015.txt::c0].

Procedure: The agent should guide the user through the "Forgot Password" or "Account Access" workflow if the item was purchased under a guest profile [Chunk: ACCOUNT/ticket_10014.txt::c0].

Support Escalation: If the automated policy fails, reach out to the team at the provided support number [Chunk: ACCOUNT/ticket_10010.txt::c0].
